In [ ]:
%cd /kaggle/working
# - Paste your ngrok token below ( ngrok_token = 'YOUR_TOKEN_HERE' ) - #

ngrok_token = ''

arguments = '--theme dark --enable-insecure-extension-access --disable-safe-unpickle --no-hashing --xformers'

if ngrok_token:
    arguments += f' --ngrok {ngrok_token} '
else:
    pass

# - Dependencies - #
!apt-get update
!apt-get install -y google-perftools
!apt -y install -qq aria2
!apt -y install wget git python3 python3-venv
# !pip uninstall -y torch torchvision torchaudio
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q pyngrok
!pip install xformers==0.0.24
!python -m pip cache purge
# !pip install insightface # ← Install if you use ControlNet or ReActor extensions - #

# - Stable Diffusion WebUI - #
!git clone https://github.com/Totsukawaii/SD-webui-forge.git
%cd SD-webui-forge


In [ ]:
# Extensions
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /kaggle/working/SD-webui-forge/extensions/a1111-sd-webui-tagcomplete
# !git clone https://github.com/zanllp/sd-webui-infinite-image-browsing /kaggle/working/SD-webui-forge/extensions/sd-webui-infinite-image-browsing
!git clone https://github.com/etherealxx/batchlinks-webui /kaggle/working/SD-webui-forge/extensions/batchlinks-webui
!git clone https://github.com/Bing-su/adetailer /kaggle/working/SD-webui-forge/extensions/adetailer

In [ ]:
!wget -c https://huggingface.co/Totsukawaii/RiotDiffusion/resolve/main/riotdiffusionXL_v2-000001.safetensors -P /kaggle/working/SD-webui-forge/models/Stable-diffusion

In [ ]:
import subprocess
import threading

def run_script(script_name, args):
    # Combine the script name and arguments into a single command
    cmd = f"python {script_name} {args}"
    
    # Run the command and capture the output
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Monitor the output
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()

# Specify your scripts and their arguments
script1_name = "launch.py"
script1_args = arguments + " --port 7680 --gpu-device-id 0"

script2_name = "launch.py"
script2_args = arguments + " --port 7681 --gpu-device-id 1"

# Create threads for each script
thread1 = threading.Thread(target=run_script, args=(script1_name, script1_args))
thread2 = threading.Thread(target=run_script, args=(script2_name, script2_args))

# Start the threads
thread1.start()
thread2.start()

# Join the threads to the main thread to wait until they finish
thread1.join()
thread2.join()
